<h1 style="text-align: center;">NTHU EE6550 Machine Learning Lecture Notes</h1>
<h4 style="text-align: center;">Yu-Chun Lo (howard.lo@nlplab.cc)</h4>

本筆記除了會記錄老師上課所講的內容，也會補充主要參考的兩本書籍跟一些網路上的資源：
- [**Foundations of Machine Learning** by Mehryar Mohri, Afshin Rostamizadeh, and Ameet Talwalkar](https://www.amazon.com/Foundations-Machine-Learning-Adaptive-Computation/dp/026201825X)
- [**Deep Learning** by Ian Goodfellow, Yoshua Bengio and Aaron Courville](http://www.deeplearningbook.org/)

---

## Part 1. Introduction

**Machine Learning** 簡單來說就是使用 **learning algorithms**，從 **training data** 中訓練出一個 **model (prediction function)**，幫我們做出準確的 **prediction**。

### Fundamental Questions

- 什麼樣的 concept 是能夠學習的？
- 在什麼 condition 下才能夠學習？
- 這些 concepts 能夠被學得多好？


### Definitions and Terminology

以垃圾信分類為例 (**spam** or **non-spam**)：
- *Examples (Items, Instances)*: 在此例代表所有現有的 emails，作為之後 training 或 testing 所用。
- *Features (attributes)*: 通常被表示成一個 **vector**。在此例代表一封 email 的 **sender name**、**message length**、**message keywords** 等用來判斷是否是垃圾信的指標。
- *Labels*: 在 **classification** 中，此例的 label 就是一封信是否為 span or non-span 的 **binary categorical labels**；在 **regression** 中就是一個 **real-valued labels** $\in \mathbb{ R }$。
- *Training sample (data)*: 從 examples 中被拿來訓練 的 subset。
- *Validation sample (data)*: 從 examples 中被拿來調整 hyperparameters 的 subset。有些 learning algorithm 會需要手動給定參數，這些參數稱為 hyperparameters，所以我們可以運用 validation sample 幫助我們找到一組好的 hyper-parameters。
- *Test sample (data)*: 從 examples 中被拿來測試 的 subset，主要用來 evaluate model prediction performance。**在訓練階段，model 不能看到 test sample**，必須跟 training sample 與 validation sample 分開。
- *Loss Function*: 量化 predicted label $y'$ 與 true label $y$ 之間的 loss，主要是拿來看 model performance 好不好，loss 越低則越好。在此例，loss function 可以被表示成以下式子：
    - 在 binary classification，用 **zero-one loss**: $L(y',y) = 1_{y' \neq y}$。當預測錯誤，即 $y' \neq y$，loss 則為 $1$，反之則 $0$。
    - 在 regression，用 **the squared loss**: $L(y',y) = (y' - y)^2 \in \mathbb{ R }$。當差距越大，loss 越高。
- *Hypothesis set*: 簡單來說就是一堆可能的 models (prediction functions)。在此例，這些 functions 將 feature vector mapping 到 label。理論上我們會從 hypothesis set 中挑選一個最好的 prediction function。

> 為什麼還要多一個 validation data 而不是只要用 test data 來測試 model performance 後，再回去調整 hyperparameters 直到 performance 表現得很好為止？

> 因為如果你只用 test data 來調整 hyperparameters 而讓 performance 變得很好，某種程度上，你是將 test data 視為你的 training data 了，而沒有真正的被 test 到，失去了 **generalization** 的能力，或是說 model 對 test data **overfitting** 了，因此 test data 必須被當成寶貴的「未知資料」，訓練階段絕不可以動到 test data，在最後關頭才拿來測試 model 的 performance。因此，為了不動到 test data，我們把 training data 的一部份抽出來當成 validation data 幫助我們調整 hyperparameters，跟一開始就切割好 validation data 也是一樣的意思。

### Learning Stages
1. 隨機將 examples 切割成 training data, validation data and test data。通常會平均切割，或 training data 會切得比 testing data 大份；validation data 要切多大就看 hyperparameters 數量有多大。
2. 這是**最重要**的一環：選擇好的 features based on user's domain knowledge 或是使用 deep learning 中的 **representation learning** 去自動學習好的 features。Features 的好壞對 performance 的影響非常大。
3. 手動給定幾組不同 的 hyperparameters，開始用 training data 中決定好的 features 當作 input 去訓練我們的 model。因為有好幾組不同的 hyperparameters 的實驗設定，因此我們就會得出好幾個不同的 models，也就是 hypothesis set。
4. 接著，我們使用 validation data，根據定義好的 loss function 去測試 hypothesis set 中所有的 models 的 performance，幫助我們找到最好的 model，也就是 loss 最低。
- 最後，我們使用 test data 再去測試我們在上一個步驟利用 validation data 挑出來最好的 model。如果 model 在 training data 表現得很好，卻在 test data 表現得很差，代表我們的 model 可能對 training data **overfitting** 了，而失去了 **generalization**。

### Cross-validation
當我們 examples 數量不足以讓我們多切割一份 validation data 時，我們可以使用 n-fold cross-validation 幫我們選出最好的 hyperparameters，在這邊 hyperparameters 表示成 vector $\theta$，以下為針對一個 fixed $\theta$ 的 cross-validation 的步驟：
1. 假設我們有 $m$ 個 training data，隨機將 training data 切割成 $n$ 個 folds，假設第 $i$ 個 fold 有 $m_i$ 個 training examples，$1 \leq i \leq n$，而 $m_i$ 通常為 $\frac{m}{n}$ 。
2. For 第 $i$ 個 fold，用其他 folds 當作 training data 拿去訓練，得到一個 hypothesis (model/prediction function) $h_i$，然後使用第 $i$ 個 fold 當作 validation data 來測試 $h_i$ 的 *cross-validation error*，公式如下：
$$ R_{cv}(\theta) = \frac{1}{n} \sum_{i=1}^{n} \frac{1}{m_i} \sum_{j=1}^{m_i} L(y'_{ij}, y_{ij}) $$

我們目標就是要選一組好的 $\theta^*$, 能夠 minimize *cross-validation error*：$\theta^* = \underset{\theta}{\mathrm{arg min}}\ R_{cv}(\theta) $

> 實務上，如果資料夠大，大家還是只會用一份 validation data，畢竟 cross-validation 的運算量很大也會花不少時間。選擇多大的 validation data 就看你有多大的 hyperparameters；如果 validation data 的 size 比較小，則還是推薦使用 cross-validation，通常切割 fold 的數量是 3, 5, 10。

### Model Selection
我們利用上述 cross-validation 取得最好的 hyperparameter $\theta^*$ 之後，我們再拿這個 $\theta^*$ 當作我們 learning algorithm 的 hyperparameter setting，並且拿**整份完整**的 training data 拿去訓練 model，再拿 test data 去測試 model 的 performance。